## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("/Users/16472/Bootcamp/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Joao Da Barra,BR,-21.6403,-41.0511,82.40,65,0,23.02,clear sky
1,1,Champerico,GT,14.3000,-91.9167,91.40,52,40,11.50,scattered clouds
2,2,Albany,US,42.6001,-73.9662,17.01,52,100,2.53,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,69.01,68,75,6.91,broken clouds
4,4,Aykhal,RU,66.0000,111.5000,-41.69,78,100,9.80,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperatue you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperatue you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] > min_temp)]
preferred_cities_df.head(10)



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Joao Da Barra,BR,-21.6403,-41.0511,82.40,65,0,23.02,clear sky
1,1,Champerico,GT,14.3000,-91.9167,91.40,52,40,11.50,scattered clouds
7,7,Lokoja,NG,7.8024,6.7430,81.82,22,38,3.09,scattered clouds
8,8,Road Town,VG,18.4167,-64.6167,78.80,78,40,10.36,scattered clouds
10,10,Souillac,MU,-20.5167,57.5167,80.60,78,40,13.80,scattered clouds
14,14,Florianopolis,BR,-27.6146,-48.5012,78.80,73,40,5.75,scattered clouds
16,16,Avarua,CK,-21.2078,-159.7750,77.00,94,75,10.36,heavy intensity drizzle
18,18,Dakar,SN,14.6937,-17.4441,75.20,46,0,11.50,clear sky
24,24,Hithadhoo,MV,-0.6000,73.0833,82.20,77,75,13.33,broken clouds
25,25,Veracruz,MX,19.3333,-96.6667,79.00,73,40,13.80,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                196
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
clean_df = preferred_cities_df.copy()

C:\Users\16472\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Joao Da Barra,BR,82.40,clear sky,-21.6403,-41.0511,
1,Champerico,GT,91.40,scattered clouds,14.3000,-91.9167,
7,Lokoja,NG,81.82,scattered clouds,7.8024,6.7430,
8,Road Town,VG,78.80,scattered clouds,18.4167,-64.6167,
10,Souillac,MU,80.60,scattered clouds,-20.5167,57.5167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    #6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print ("Hotel not found.... skipping.")

Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
hotel_df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)

clean_hotel_df = hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng", "Hotel Name"]].copy()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Joao Da Barra,BR,82.40,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
1,Champerico,GT,91.40,scattered clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
7,Lokoja,NG,81.82,scattered clouds,7.8024,6.7430,Hotel Charvid
8,Road Town,VG,78.80,scattered clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
10,Souillac,MU,80.60,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
...,...,...,...,...,...,...,...
681,Villa Bruzual,VE,83.12,overcast clouds,9.3328,-69.1208,Edificio Guarente
684,Ilhabela,BR,77.59,broken clouds,-23.7781,-45.3581,Ilha Flat Hotel
685,Tena,EC,86.00,broken clouds,-0.9833,-77.8167,Hostal Tena Ñaui
686,Kavaratti,IN,80.92,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt>{Current Description}</dd>
<dt>Max Temp</dt>{Max Temp}</dd>
</dt>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))